# Coursework 3: Reed-Solomon Codes

[] By tick the checkbox, we hereby declare that this coursework report is our own and autonomous work. We have acknowledged all material and sources used in its preparation, including books, articles, reports, lecture notes, internet software packages, and any other kind of document, electronic or personal communication. This work has not been submitted for any other assessment.

**Julia Packages**

We use the `LinearAlgebra` package for matrix operations.
We use the `StatsBase` package for introducing random error.

In [109]:
using Pkg
Pkg.add("LinearAlgebra")
Pkg.add("StatsBase")

using LinearAlgebra
using StatsBase

   Resolving package versions...
  No Changes to `C:\Users\aydes\.julia\environments\v1.6\Project.toml`
  No Changes to `C:\Users\aydes\.julia\environments\v1.6\Manifest.toml`
   Resolving package versions...
  No Changes to `C:\Users\aydes\.julia\environments\v1.6\Project.toml`
  No Changes to `C:\Users\aydes\.julia\environments\v1.6\Manifest.toml`


#### Helper Functions

In [110]:
# Helper Functions

# Function to do modulo division
function modulo_div(num,den,p)
    return mod((num * gcdx(den,p)[2]),p)
end

# Function for modular exoponentiation
function modExpo(p,x,n)
    if p == 1 
        return 0
    end
    a = 1 # initialise a
    binaryExp = mod(x,p) #binaryExp when i=0
    while n > 0 #loop cycles through each binary digit of n - (cycle of ceiling[log2(n)] times)
        if (mod(n,2) == 1) #check LSB
            a = mod((a * binaryExp), p) #if LSB is 1 then update a accordingly
        end
        n = n >> 1 #right shift exponent to check next bit
        binaryExp = mod((binaryExp * binaryExp), p) #calculate new binaryExp value by squaring previous result and taking modulus
    end
    return a
end

modExpo (generic function with 1 method)

#### Custom Polynomial Struct/Class

In order to make handling and operating on polynomials easier, we have written a **very extensive** struct for polynomials, making use of Julia's type templates and multiple dispatch (operator overloading) functionality.
A polynomial is of type `T` when its coefficients are of type `T`. In this coursework, we encounter two forms of polynomials. For the first half of the coursework, where we work on $\mathbb{F}_p$, the coefficeints of the polynomial are integers. Therefore we have `MyPolynomial{Int}`. 
For the second half, we work on $\mathbb{F}_{p^k}$, so we also encounter  `MyPolynomial{MyPolynomial{Int}}` as the polynomials are the coefficients of another polynomial.

A Polynomial is defined by `struct MyPolynomial` by:
 - `coeffs`: A vector of the polynomials coefficients, listed in ascending order of the power of $x$ they multiply. This means that $5x^2 + 3x + 2$ would be represented by [2,3,5]
 - `order`: The order of the polynomial is the highest power of $x$ in the polynomial. Does not have to be supplied to the constructor, instead it is automatically worked out.
 - `p`: All polynomial operations occur within a finite field. Therefore, it is wise to store $p$, as this helps during operations.

In [111]:
# Struct Definition for MyPolynomial
struct MyPolynomial{T}
    coeffs::Vector{T}
    order::Int
    p::Int
end

"""
ConstructMyPolynomial(coeffs,p)

Info: 
Outer Constructor function for the MyPolynomial Struct

Args:
coeffs: An array of the polynomial's coefficients listed in ascending order of the power of x they multiply.
p: The p that defines the Finite Field that contains the coefficients.

Returns:
MyPolynomial{T} object.
    
"""
function ConstructMyPolynomial(coeffs,p)
    coeffs = [mod(i,p) for i in coeffs]
    return MyPolynomial(coeffs,length(coeffs) - 1,p)
end


"""
remove_top_zeros(poly::MyPolynomial)

Info:
Function to remove extra zeros from the top of a polynomial's coefficient array.
For example: [2,3,5,0,0] --> [2,3,5]
Has the additional use of acting like a deep_copy for a polynomial object, as it returns a new object.

Args:
poly: A MyPolynomial object whose coefficient array length needs to be corrected.

Returns:
MyPolynomial{T} object with a coefficient array where length(coeffs) - 1 = order of the polynomial.
"""
function remove_top_zeros(poly::MyPolynomial)
    p = poly.p
    coeffs = poly.coeffs
    if typeof(poly) == MyPolynomial{Int}
        zero_term = 0
    else
        zero_term = ConstructMyPolynomial([0],p)
    end
    while length(coeffs) > 0 && last(coeffs) == zero_term
        pop!(coeffs)
    end
    if length(coeffs) == 0
        push!(coeffs,zero_term)
    end
    return ConstructMyPolynomial(coeffs,poly.p)
end


"""
make_same_length(poly1::MyPolynomial{Int}, poly2::MyPolynomial{Int})

Info:
Function to make two polynomials with integer coefficients the same length.
This is achieved by pushing zeros on to the top of the lower-order polynomial.

Args:
poly1,poly2: Two MyPolynomial{Int} objects

Returns:
Two MyPolynomial{Int} objects, both with coefficent arrays of the same length.
"""
function make_same_length(poly1::MyPolynomial{Int}, poly2::MyPolynomial{Int})
    len1 = length(poly1.coeffs)
    len2 = length(poly2.coeffs)

    if len1 > len2
        zero_pad = zeros(Int,len1-len2)
        new_coeffs = cat(poly2.coeffs,zero_pad,dims=1)
        new_poly = ConstructMyPolynomial(new_coeffs,poly2.p)
        return (poly1,new_poly)
    else
        zero_pad = zeros(Int,len2-len1)
        new_coeffs = cat(poly1.coeffs,zero_pad,dims=1)
        new_poly = ConstructMyPolynomial(new_coeffs,poly1.p)
        return (poly2,new_poly)

    end
end


"""
make_same_length(poly1::MyPolynomial{MyPolynomial{Int}}, poly2::MyPolynomial{MyPolynomial{Int}})

Info:
Identical function as the previous one, but overloaded for handling polynomials with polynomial coefficients
"""
function make_same_length(poly1::MyPolynomial{MyPolynomial{Int}}, poly2::MyPolynomial{MyPolynomial{Int}})
    p = poly1.p
    len1 = length(poly1.coeffs)
    len2 = length(poly2.coeffs)

    if len1 > len2
        zero_pad = zeros_poly(len1-len2,p)
        new_coeffs = cat(poly2.coeffs,zero_pad,dims=1)
        new_poly = ConstructMyPolynomial(new_coeffs,poly2.p)
        return (poly1,new_poly)
    else 
        zero_pad = zeros_poly(len2-len1,p)
        new_coeffs = cat(poly1.coeffs,zero_pad,dims=1)
        new_poly = ConstructMyPolynomial(new_coeffs,poly1.p)
        return (new_poly,poly2)
    end
end


"""
Base.:mod(poly::MyPolynomial,p::Int)

Info:
Overloaded function to mod a polynomial with a number.
Essentially mods each coefficient of the polynomial with p.

Args:
poly: MyPolynomial object to be modded
p: Integer which the polynomial will be modded by

Returns:
MyPolynomial object, with coefficients in Fₚ
"""
function Base.:mod(poly::MyPolynomial,p::Int)
    new_coeffs = [mod(i,p) for i in poly.coeffs]
    return ConstructMyPolynomial(new_coeffs,p)
end


"""
Base.:mod(poly1::MyPolynomial,poly2::MyPolynomial)

Info:
Overloaded function to mod a polynomial with another polynomial.

Args:
poly1: MyPolynomial object which is the LHS of the mod.
poly2: MyPolynomial object which is the RHS of the mod.

Returns:
MyPolynomial object, with coefficients in Fₚ/f(x), where f(x) is poly2.
"""
function Base.:mod(poly1::MyPolynomial,poly2::MyPolynomial)
    return divrem(poly1,poly2)[2]
end

function Base.:mod(poly1::MyPolynomial{MyPolynomial{Int}},poly2::MyPolynomial{Int})
    p = poly1.p
    new_coeffs = [mod(i,poly2) for i in poly1.coeffs]
    return ConstructMyPolynomial(new_coeffs,p)
end


"""
Base.:length(poly::MyPolynomial)

Overloaded funciton which returns the length of a polynomial's coefficient array.
"""
function Base.:length(poly::MyPolynomial)
    return length(poly.coeffs)
end


"""
modulo_div(poly1::MyPolynomial{Int},poly2::MyPolynomial{Int},f::MyPolynomial{Int})

Info:
Function to perform modulo division of two polynomials within a Finite Field Fₚ/f(x). p is inferred from the polynomial.

Args:
poly1: MyPolynomial{Int} object, which is the numerator.
poly2: MyPolynomial{Int} object, which is the denominator.
f: f(x), the polynomial that defines the finite field.
"""
function modulo_div(poly1::MyPolynomial{Int},poly2::MyPolynomial{Int},f::MyPolynomial{Int})
    mul_inv = gcdx(poly2,f)[2]
    return mod(poly1 * mul_inv,f)
end


"""
zeros_poly(num::Int, p::Int)

Info:
Equivilent to the Base.zeros() function in Julia, but for polynomials.
"""
function zeros_poly(num::Int, p::Int)
    zero_poly = ConstructMyPolynomial([0],p)
    z = []
    for i in 1:num
        push!(z,zero_poly)
    end
    return z
end

"""
ones_poly(num::Int, p::Int)

Info:
Equivilent to the Base.ones() function in Julia, but for polynomials.
"""
function ones_poly(num::Int, p::Int)
    one_poly = ConstructMyPolynomial([1],p)
    z = []
    for i in 1:num
        push!(z,one_poly)
    end
    return z
end


"""
Base.:fill(poly::MyPolynomial, num::Int)

Info:
Overloaded function for Base.fill() for polynomials
"""
function Base.:fill(poly::MyPolynomial, num::Int)
    poly = remove_top_zeros(poly)
    z::Vector{MyPolynomial} = []
    for i in 1:num
        push!(z,poly)
    end
    return z
end

#========================================================
OVERLOADED ARITHMETIC OPERATORS/FUNCTIONS FOR POLYNOMIALS
=========================================================#

function Base.:+(poly1::MyPolynomial, poly2::MyPolynomial)
    p = poly1.p
    poly1 = remove_top_zeros(poly1)
    poly2 = remove_top_zeros(poly2)

    poly1,poly2 = make_same_length(poly1,poly2)

    new_coeffs = poly1.coeffs + poly2.coeffs
    return remove_top_zeros(ConstructMyPolynomial(new_coeffs,p))
end

function Base.:*(poly::MyPolynomial, num::Int)
    p = poly.p
    new_coeffs = [mod(i,p) for i in (poly.coeffs * num)]
    return remove_top_zeros(ConstructMyPolynomial(new_coeffs,p))
end

function Base.:*(num::Int, poly::MyPolynomial)
    return poly * num
end

function Base.:-(poly1::MyPolynomial, poly2::MyPolynomial)
    return poly1 + (-1 * poly2)
end

function Base.:*(poly1::MyPolynomial{Int}, poly2::MyPolynomial{Int})
    p = poly1.p
    poly1 = remove_top_zeros(poly1)
    poly2 = remove_top_zeros(poly2)

    c1 = poly1.coeffs
    c2 = poly2.coeffs
    m = length(c1)
    n = length(c2)

    new_coeffs = zeros(Int,m+n-1)
    for i in 1:m
        for j in 1:n
            new_coeffs[i+j-1] += c1[i]*c2[j]
        end
    end

    return remove_top_zeros(ConstructMyPolynomial(new_coeffs,p))
end

function Base.:*(poly1::MyPolynomial{MyPolynomial{Int}}, poly2::MyPolynomial{MyPolynomial{Int}})
    p = poly1.p
    poly1 = remove_top_zeros(poly1)
    poly2 = remove_top_zeros(poly2)

    c1 = poly1.coeffs
    c2 = poly2.coeffs
    m = length(c1)
    n = length(c2)

    new_coeffs = zeros_poly(m+n-1,p)
    for i in 1:m
        for j in 1:n
            new_coeffs[i+j-1] += c1[i]*c2[j]
        end
    end

    return remove_top_zeros(ConstructMyPolynomial(new_coeffs,p))
end

function Base.:*(poly1::MyPolynomial{MyPolynomial{Int}},poly2::MyPolynomial{Int})
    p = poly1.p
    poly1 = remove_top_zeros(poly1)
    poly2 = remove_top_zeros(poly2)
    
    new_coeffs = [mod(i*poly2,p) for i in poly1.coeffs]
    return remove_top_zeros(ConstructMyPolynomial(new_coeffs,p))
end


function Base.:^(poly::MyPolynomial, num::Int)
    if num == 0
        return ConstructMyPolynomial([1],poly.p)
    end
    
    poly_acc = ConstructMyPolynomial(poly.coeffs,poly.p)
    for i in 2:num
        poly_acc = poly_acc * poly
    end
    return poly_acc
end

function Base.:/(poly1::MyPolynomial{Int},poly2::MyPolynomial{Int})
    return modulo_div(poly1,poly2,ConstructMyPolynomial([1],poly1.p))
end

function Base.:divrem(poly1::MyPolynomial{Int}, poly2::MyPolynomial{Int})
    p = poly1.p
    poly1 = remove_top_zeros(poly1)
    poly2 = remove_top_zeros(poly2)

    num = poly1.coeffs
    den = poly2.coeffs

    if length(num) >= length(den)
        # Shift denominator to the right to match degree of numerator.
        shiftamount = length(num) - length(den)
        den = append!(zeros(Int,shiftamount),den)
    else
        # This is the case where degree of denominator > degree of numerator.
        # Results in zero quotient and the numerator being the remainder.
        return (ConstructMyPolynomial([0],p),remove_top_zeros(ConstructMyPolynomial(num,p)),)
    end

    quotient = []
    divisor = last(den)

    for i in 1:(shiftamount+1)
        #Get the next coefficient of the quotient.
        mult = modulo_div(last(num),divisor,p)
        quotient = append!([mult],quotient)
        
        if mult != 0
            d = [mult * u for u in den]
            for i in 1:length(num)
                num[i] = mod((num[i] - d[i]),p)
            end
        end
        
        pop!(num)
        popfirst!(den)
    end
    return (remove_top_zeros(ConstructMyPolynomial(quotient,p)),remove_top_zeros(ConstructMyPolynomial(num,p)))
end

function Base.:divrem(poly1::MyPolynomial{MyPolynomial{Int}}, poly2::MyPolynomial{MyPolynomial{Int}},f::MyPolynomial{Int})
    p = poly1.p
    poly1 = remove_top_zeros(poly1)
    poly2 = remove_top_zeros(poly2)

    num = poly1.coeffs
    den = poly2.coeffs

    if length(num) >= length(den)
        # Shift denominator to the right to match degree of numerator.
        shiftamount = length(num) - length(den)
        den = append!(zeros_poly(shiftamount,p),den)
    else
        # This is the case where degree of denominator > degree of numerator.
        # Results in zero quotient and the numerator being the remainder.
        return (ConstructMyPolynomial([0],p),remove_top_zeros(ConstructMyPolynomial(num,p)))
    end

    quotient = []
    divisor = last(den)

    for i in 1:(shiftamount+1)
        #Get the next coefficient of the quotient.
        mult = modulo_div(last(num),divisor,f) # Should be modulo division with respect to F[x]/f(x)
        quotient = append!([mult],quotient)
        
        if mult != 0
            d = [mod(mult*u,f) for u in den]
            for i in 1:length(num)
                num[i] = mod((num[i] - d[i]),f)
            end
        end
        
        pop!(num)
        popfirst!(den)
    end
    return (remove_top_zeros(ConstructMyPolynomial(quotient,p)),remove_top_zeros(ConstructMyPolynomial(num,p)),)
end

function Base.:divrem(poly1::MyPolynomial{MyPolynomial{Int}},poly2::MyPolynomial{Int})
    p = poly1.p
    poly1 = remove_top_zeros(poly1)
    poly2 = remove_top_zeros(poly2)
    
    new_coeffs = [divrem(i,poly2)[1] for i in poly1.coeffs]
    return remove_top_zeros(ConstructMyPolynomial(new_coeffs,p))
end

function Base.:powermod(poly::MyPolynomial, num::Int, f::MyPolynomial)
    if num == 0
        return ConstructMyPolynomial([1],poly.p)
    end
    f = remove_top_zeros(f)
    poly_acc = remove_top_zeros(poly)
    for i in 2:num
        poly_acc = divrem(poly_acc * poly,f)[2]
    end
    return poly_acc
end

function Base.:gcdx(poly1::MyPolynomial{Int},poly2::MyPolynomial{Int})
    p = poly1.p
    swapped = false
    if poly1.order < poly2.order
        swapped = true
        a = remove_top_zeros(poly2)
        b = remove_top_zeros(poly1)
    else
        a = remove_top_zeros(poly1)
        b = remove_top_zeros(poly2)
    end

    x0,x1 = ConstructMyPolynomial([1],p),ConstructMyPolynomial([0],p)
    y0,y1 = ConstructMyPolynomial([0],p),ConstructMyPolynomial([1],p)

    if b == ConstructMyPolynomial([0],p)
        if swapped 
            return (a,y0,x0)
        else
            return (a,x0,y0)
        end
    end

    r = b

     while remove_top_zeros(r) != ConstructMyPolynomial([0],p)
        q,r = divrem(a,b)
        a = b
        b = r

        x0, x1 = x1, x0 - q*x1
        y0, y1 = y1, y0 - q*y1
    end

    if swapped
        return (a,y0,x0)
    else
        return (a,x0,y0)
    end
end

function Base.:zero(x::MyPolynomial{Int})
    return ConstructMyPolynomial([0],x.p)
end

function Base.:one(x::MyPolynomial{Int})
    return ConstructMyPolynomial([1],x.p)
end

#================================================
OVERLOADED LOGICAL OPERATORS FOR POLYNOMIALS
=================================================#

function Base.:(==)(poly1::MyPolynomial,poly2::MyPolynomial)
    return poly1.coeffs == poly2.coeffs
end

function Base.:(!=)(poly1::MyPolynomial,poly2::MyPolynomial)
    return poly1.coeffs != poly2.coeffs
end

#================================================#

"""
derivative(poly::MyPolynomial)

Info:
Function to find the derivative of the supplied polynomial with respect to the main varaible of the polynomial.
Note: Even for polynomials with polynomial coefficients, this function will treat them as coefficients of a different variable,
meaning that it will not differentiate with respect to them (treating them like it would treat an integer).

Args:
poly: MyPolynomial object to be differentiated.

Returns:
MyPolynomial object, which is the first derivitive of the input.

"""
function derivative(poly::MyPolynomial)
    coeffs = poly.coeffs
    p = poly.p
    new_coeffs = []

    for i in 2:length(coeffs)
        push!(new_coeffs,(i-1)*coeffs[i])
    end
    return remove_top_zeros(ConstructMyPolynomial(new_coeffs,p))
end


"""
evaluate(poly::MyPolynomial{Int},x::Int)

Info:
Function to evaluate a polynomial at a given value.

Args:
poly: MyPolynomial object to be evaluated.
x: The x value at which the polynomial is to be evauluated

Returns:
Integer value of the polynomial at the given input (in Fₚ)

"""
function evaluate(poly::MyPolynomial{Int},x::Int)
    coeffs = poly.coeffs
    p = poly.p
    val = 0
    for i in 1:length(coeffs)
        val = mod(val + coeffs[i] * powermod(x,(i-1),p),p)
    end
    return mod(val,p)
end


"""
evaluate(poly1::MyPolynomial{MyPolynomial{Int}},poly2::MyPolynomial{Int},f::MyPolynomial{Int})

Info:
Function to evaluate a polynomial with polynomial coefficients at a given polynomial value.

Args:
poly1: MyPolynomial object to be evaluated.
poly2: The polynomial to be substituted in.
f: The f(x) that defines the Finite Field Fₚ/f(x).

Returns:
MyPolynomial object, which is the value of the polynomial at the given input (in Fq)

"""
function evaluate(poly1::MyPolynomial{MyPolynomial{Int}},poly2::MyPolynomial{Int},f::MyPolynomial{Int})
    coeffs = poly1.coeffs
    p = poly1.p
    val = ConstructMyPolynomial([0],p)
    for i in 1:length(coeffs)
        val = mod(val + mod((coeffs[i] * powermod(poly2,i-1,f)),f),f)
    end
    return remove_top_zeros(mod(val,f))
end


"""
Base.:string(poly::MyPolynomial{MyPolynomial{Int}},var::Char)

Info:
Overloaded function to convert a polynomial to a string in preparation for printing.

Args:
poly: MyPolynomial object to be converted to string.
var: A char representing the chosen variable to be printed. (e.g. 'x' --> 1 + x + x^2 | 'z' --> 1 + z + z^2)

Returns:
A string representing the polynomial in readable format.
"""
function Base.:string(poly::MyPolynomial{MyPolynomial{Int}},var::Char)
    poly = remove_top_zeros(poly)
    coeffs = poly.coeffs
    i = 0
    s = string(first(coeffs),var)
    for i in 2:length(coeffs)
        s = string(s," + [",string(coeffs[i],var),"]",var,"^",i-1)
    end
    return s
end

# Same as the above but for MyPolynomial{Int}
function Base.:string(poly::MyPolynomial{Int},var::Char)
    poly = remove_top_zeros(poly)
    coeffs = poly.coeffs
    i = 0
    s = string(first(coeffs))
    for i in 2:length(coeffs)
        s = string(s," + (",coeffs[i],var,"^",i-1,")")
    end
    return s
end

"""
Base.:print(poly::MyPolynomial,var::Char)

Info:
Overloaded function to print a polynomial in a human-readable format.

Args:
poly: MyPolynomial object to be printed.
var: A char representing the chosen variable to be printed. (e.g. 'x' --> 1 + x + x^2 | 'z' --> 1 + z + z^2)
"""
function Base.:print(poly::MyPolynomial,var::Char)
    println(string(poly,var))
end


Base.print

## 3.1 RS Codes on $\mathbb{F}_p$ (50%)

### 3.1.1 RS Codes on $\mathbb{F}_{17}$

1. In $\mathbb{F}_{17}$, find a primitive element $\alpha$. 
2. Let $\mathcal{C} \subset \mathbb{F}_{17}^{16}$ be an RS code with parameters $[16,k,7]$. What is the value of $k$? 
3. Design and implement an encoding function `RSNum_Enc` that encodes a message $\boldsymbol{m} \in \mathbb{F}_{17}^k$ to a codeword $\boldsymbol{c} \in \mathcal{C}$. 
4. Let $\boldsymbol{y} = \boldsymbol{c} + \boldsymbol{e}$ be the received word where the channel introduces $t = \lfloor \frac{d-1}{2}\rfloor$ errors in random places. Design and implement a function `RSNum_Channel` for this. Julia function `StatsBase > Sample` can be helpful in randomly chose error locations.
5. Design and implement a function `RSNum_Syndrome` to calculate the syndrome vector and the syndrome polynomial from the received word $\boldsymbol{y}$. 
6. Design and implement a function `RSNum_KeyPolys` to find error locator polynomial $L(z)$ and error evaluator polynomial $E(z)$ from the syndrome polynomial $S(z)$. 
7. Design and implement a function `RSNum_Err` to find the estimated error vector $\hat{\boldsymbol{e}}$ and the estimated codeword $\hat{\boldsymbol{c}}$.
8. Design and implement a function `RSNum_Message` to find the estimated message $\hat{\boldsymbol{m}}$. 
9. Implement a function `RSNum_Dec` which uses the functions in Steps 5-8 as sub-routines for decoding.  

Provide necessary documentation.

#### 1. Find a Primitive Element $\alpha$

We use the function `find_primitve(p)`, which returns an array of all primitive elements in the finite field. We then pick one at random to be our $\alpha$.

In [112]:
# Utility function for find_primitive
# Checks if an element is a primitive element of finite field Fₚ
function is_primitive_element(element,p)
    if modExpo(p,element,p-1) != 1
        return false
    end

    # Using ord(β) | p-1 for Fₚ.
    # Using p instead of q as p = q for integer ring finite fields.
    # We only need to check powers that are factors of p-1.
    factors = []
    for i in 2:isqrt(p-1)
        quot,rem = divrem(p-1,i)
        if rem == 0
            append!(factors,[i,quot])
        end
    end
    
    for t in factors
        if modExpo(p,element,t) == 1
            return false
        end
    end
    return true
end


"""
find_primitive(p)

Info:
Finds primitve elements for the Finite Field Fₚ denoted by input p.

Args:
p: Prime number defining Finite Field Fₚ.

Returns:
A vector of integers which are elements of Fₚ.
"""
function find_primitive(p)
    field = collect(0:p-1)
    prim_els = []
    for a in field
        if is_primitive_element(a,p)
            push!(prim_els,a)
        end
    end
    return prim_els
end

find_primitive(17)

8-element Vector{Any}:
  3
  5
  6
  7
 10
 11
 12
 14

In [113]:
# Define the prime number for our Finite Field
p = 17

# Get all primitive elements in that Finite Field
prim_els = find_primitive(p)

# Pick a primitive element at random to be α
α = prim_els[rand(1:length(prim_els))]
@show α

α = 14


14

#### 2. What is the value of $k$?

RS Codes are defined by three parameters: **$[n,k,d]$**, where $d = n - k + 1$.

Taking the RS Code given in the question:  **$[16,k,7]$**, we can see that $n = 16$ and $n - k + 1 = 7$. 

We can substitute in $n = 16$ to get: $16 - k + 1 = 7$. Solving this equation yields **$k = 10$**.

In [114]:
# Define the values of [n,k,d]
n = 16
k = 10
d = 7

7

#### 3. RS Encoding

In [115]:
# Helper Functions for RS Encoding

"""
generator_matrix(p,α,n,k)

Info:
Creates the generator matrix, G, for RS Encoding using the given arguments.

Args:
p: The prime number that defines the Finite Field Fₚ
α: The chosen primitive element from Fₚ
n: The length of the code
k: The length of the message

Returns:
A k×n matrix of type Matrix{Int} which is the generator matrix of the RS Code [n,k,d]
"""
function generator_matrix(p,α,n,k)
    # First row of the matrix will always be n 1s, so generate that
    gen_mat = ones(Int,1,n)
    
    for i in 1:k-1
        # Calculate the defining element for that row (αⁱ, where i goes up to k)
        αⁱ = modExpo(p,α,i)
        row = fill(αⁱ,1,n)
        for j in 1:n
            # Mulitply the defining element for the given row by ascending powers up to n
            row[j] = modExpo(p,row[j],j-1)
        end
        # Add the new row to the bottom of the matrix
        gen_mat = vcat(gen_mat,row)
    end
    return gen_mat
end


generator_matrix

In [116]:
"""
RSNum_Enc(message,p,α,n,k)

Info:
Encodes the given message using Reed-Solomon Encoding.

Args:
message: Matrix{Int} of dimension 1×k containing the message
p: The prime number that defines the Finite Field Fₚ
α: The chosen primitive element from Fₚ
n: The length of the code
k: The length of the message

Returns:
A 1×n matrix of type Matrix{Int}, which is the encoded message
"""

function RSNum_Enc(message,p,α,n,k)
    if typeof(message) != Matrix{Int}
        println("Please enter message in the format of a 1×k matrix")
        println("Type of message should be Matrix{Int}, not $(typeof(message))")
        return -1
    end
    if length(message) != k
        println("Length of message should be $k, which is the value of k provided")
        return -1
    end

    # Ensure each element in our message is in Fₚ
    # If not then, perform modulo to make it comply
    message = [mod(i,p) for i in message]

    # Calculate the Generator Matrix for the given parameters
    G = generator_matrix(p,α,n,k)

    # Calculate the code by multiplying the message and the generator matrix
    code = message * G
    code = [mod(i,p) for i in code]

    return code

end

RSNum_Enc (generic function with 1 method)

In [117]:
# Test: Encode a message
message = [7 3 8 7 1 9 4 4 0 0]
codeword = RSNum_Enc(message,p,α,n,k)

1×16 Matrix{Int64}:
 9  1  15  16  9  5  10  8  14  14  16  12  0  9  0  8

#### 4. Simulating the Channel

In [118]:
"""
RSNum_Channel(code,p,n,d)

Info:
Simulates errors in the channel by adding (d-1)/2 errors, so that we can test our error correction mechanism later.
This is achieved by randomly choosing the position of the errors, and then adding a random number to the code in that position.

Args:
code: Matrix{Int} of dimension 1×n, the codeword to be transmitted accross the channel
p: The prime number that defines the Finite Field Fₚ
n: The length of the code
d: d as defined for the RS Code

Returns:
A 1×n matrix of type Matrix{Int}, which is the codeword with error introduced
"""

function RSNum_Channel(code,p,n,d)
    # Maximum number of errors
    t::Int = floor((d-1)/2)

    # Randomly choose t unique locations of the errors
    I = sample((1:n),t,replace=false)
    I_for_show = [i-1 for i in I]
    @show I_for_show
    # Build error vector, which has non-zero elements at eᵢ where i∈I
    e = []
    for i in 1:n
        if i in I
            push!(e,rand(1:p))
        else
            push!(e,0)
        end
    end

    # Convert e from type Vector{Int} to Matrix{Int}
    # This makes it the same type and shape as the code, allowing for addition
    e = reshape(e,1,n)
    @show e
    # Add error to the code
    changed_code = code + e
    changed_code = [mod(i,p) for i in changed_code]

    return changed_code
end


RSNum_Channel (generic function with 1 method)

In [119]:
# Test: Transmit codeword through channel
received = RSNum_Channel(codeword,p,n,d)

I_for_show = [7, 5, 13]
e = Any[0 0 0 0 0 11 0 14 0 0 0 0 0 15 0 0]


1×16 Matrix{Int64}:
 9  1  15  16  9  16  10  5  14  14  16  12  0  7  0  8

#### 5. Syndrome Vector and Polynomial

In [120]:
# Helper functions

"""
parity_check_matrix(p,α,n,k)

Info:
Creates the parity-check matrix, H, for RS Decoding using the given arguments.

Args:
p: The prime number that defines the Finite Field Fₚ
α: The chosen primitive element from Fₚ
n: The length of the code
k: The length of the message

Returns:
A (n-k)×n matrix of type Matrix{Int} which is the parity-check matrix of the RS Code [n,k,d]
"""
function parity_check_matrix(p,α,n,k)
    pc_mat = zeros(Int,n-k,n)
    for i in 1:n-k
        # Calculate the defining element for that row (αⁱ, where i goes up to k)
        αⁱ = modExpo(p,α,i)
        row = fill(αⁱ,1,n)
        for j in 1:n
            # Multiply the defining element for the given row by ascending powers up to n
            pc_mat[i+(j-1)*(n-k)] = modExpo(p,row[j],j-1)
        end
    end
    return pc_mat
end

parity_check_matrix(p,α,n,k)


6×16 Matrix{Int64}:
 1  14   9   7  13  12  15   6  16   3   8  10   4   5   2  11
 1   9  13  15  16   8   4   2   1   9  13  15  16   8   4   2
 1   7  15   3   4  11   9  12  16  10   2  14  13   6   8   5
 1  13  16   4   1  13  16   4   1  13  16   4   1  13  16   4
 1  12   8  11  13   3   2   7  16   5   9   6   4  14  15  10
 1  15   4   9  16   2  13   8   1  15   4   9  16   2  13   8

In [121]:
# Verifying that the identity GHᵀ = 0 holds
# Done to test generator_matrix and parity_check_matrix functions
G = generator_matrix(p,α,n,k)
H = parity_check_matrix(p,α,n,k)

A = G * transpose(H)
A = [mod(i,p) for i in A]


10×6 Matrix{Int64}:
 0  0  0  0  0  0
 0  0  0  0  0  0
 0  0  0  0  0  0
 0  0  0  0  0  0
 0  0  0  0  0  0
 0  0  0  0  0  0
 0  0  0  0  0  0
 0  0  0  0  0  0
 0  0  0  0  0  0
 0  0  0  0  0  0

In [122]:
"""
RSNum_Syndrome(changed_code,p,α,n,k)

Info:
Function to calculate the syndrome vector and syndrome polynomial from the changed code.
This is achieved by doing rHᵀ, where r is the received errorred code, and H is the parity check matrix.
r = (c + e), so the operation is actually (c + e)Hᵀ = cHᵀ + eHᵀ (by associativity of matrix multiplication).
cHᵀ = 0 by the properties of the parity-check matrix, leaving us with only eHᵀ, which is the syndrome vector.

Args:
changed_code: The received codeword, which contains errors.
p: The prime number that defines the Finite Field Fₚ
α: The chosen primitive element from Fₚ
n: The length of the code
k: The length of the message
"""
function RSNum_Syndrome(changed_code,p,α,n,k)
    # Calculate the parity-check Matrix
    H = parity_check_matrix(p,α,n,k)

    # Calculate syndrome vector
    syndrome_vector = changed_code * transpose(H)
    syndrome_vector = [mod(i,p) for i in syndrome_vector]

    # Return the syndrome vector and syndrome polynomial
    return (syndrome_vector,ConstructMyPolynomial(vec(syndrome_vector),p))
end

RSNum_Syndrome

In [123]:
# Test: Calculate Syndrome Vector/Polynomial from Received Codeword
s_vector,s_poly = RSNum_Syndrome(received,p,α,n,k)
println("Syndrome Vector: ", s_vector)
println("Syndrome Polynomial S(z) = ", string(s_poly,'z'))

Syndrome Vector: [2 15 5 3 1 11]
Syndrome Polynomial S(z) = 2 + (15z^1) + (5z^2) + (3z^3) + (1z^4) + (11z^5)


#### 6. Error Locator Polynomial and Error Evaluator Polynomial

In [124]:
# HELPER FUNCTIONS


"""
partial_euclidean_algorithm(a::MyPolynomial{Int},b::MyPolynomial{Int},stop::Int)

Info:
Helper function that performs the Extended Euclidian Algorithm on two polynomials until a specified stop point.
This stop point refers to the point at which the order of the remainder is ≤ stop.
"""
function partial_euclidean_algorithm(a::MyPolynomial{Int},b::MyPolynomial{Int},stop::Int)
    # Clean up polynomials
    p = a.p
    a = remove_top_zeros(a)
    b = remove_top_zeros(b)

    # Define initial Bezout Coefficient helpers
    x0,x1 = ConstructMyPolynomial([1],p),ConstructMyPolynomial([0],p)
    y0,y1 = ConstructMyPolynomial([0],p),ConstructMyPolynomial([1],p)

    if b == ConstructMyPolynomial([0],p)
        return (a,x0,y0)
    end

    r = b

    # Last Run boolean is required to run the algorithm one extra time to correctly calculate Bezout Coeffs.
    last_run = false

    # Do the Extended Euclidian Algorithm
     while r.order > stop || !last_run
        if r.order <= stop
            last_run = true
        end
        q,r = divrem(a,b)
        a = b
        b = r

        x0, x1 = x1, x0 - q*x1
        y0, y1 = y1, y0 - q*y1
    end
    return (a,x0,y0)
end


partial_euclidean_algorithm

In [125]:
"""
RSNum_KeyPolys(s_poly,p,α,n,d)

Info:
Function to calculate the Error Locator Polynomial (L), and the Error Evaluator Polynomial (E),
from the syndrome polynomial.

Args:
s_poly: Syndrome Polynomial
p: The prime number that defines the Finite Field Fₚ
α: The chosen primitive element from Fₚ
n: The length of the code
d: Distance

Returns:
L: MyPolynomial object representing the Error Locator Polynomial
E: MyPolynomial object representing the Error Evaluator Polynomial
"""
function RSNum_KeyPolys(s_poly,p,α,n,d)
    # Define t, last 
    t::Int = floor((d-1)/2)
    z = ConstructMyPolynomial(append!(zeros(Int,d-1),[1]),p)
    r,b,a = partial_euclidean_algorithm(z,s_poly,t-1)
    E_prime = divrem(r,z)[2]
    L_prime = divrem(a,z)[2]

    c_inv = modulo_div(1,last(L_prime.coeffs),p)

    L = L_prime * c_inv
    E = E_prime * c_inv

    @show L
    @show E

    return L,E
end


RSNum_KeyPolys

In [126]:
L,E = RSNum_KeyPolys(s_poly,p,α,n,d)

L = MyPolynomial{Int64}([11, 2, 14, 1], 3, 17)
E = MyPolynomial{Int64}([5, 16, 11], 2, 17)


(MyPolynomial{Int64}([11, 2, 14, 1], 3, 17), MyPolynomial{Int64}([5, 16, 11], 2, 17))

#### 7. Estimating Error

In [127]:
# HELPER FUNCTIONS

# Function to find the set of error locations, I.
function error_locations(L,p,α,n)
    I = []
    for k in 0:n-1
        αᵏ = powermod(α,k,p)
        # We evaluate L(α⁻ᵏ). If it equals zero we know there is an error at that location.
        if evaluate(L,modulo_div(1,αᵏ,p)) == 0
            push!(I,k)
        end
    end
    return I
end

# Function to find the estimated error vector, ê.
function error_values(E,I,p,α,n)
    # 
    dL = derivative(L)
    ê = []
    for k in 0:n-1
        if k in I
            αᵏ = powermod(α,k,p)
            α⁻ᵏ = modulo_div(1,αᵏ,p)
            E_of_α⁻ᵏ = evaluate(E,α⁻ᵏ)
            dL_of_α⁻ᵏ = evaluate(dL,α⁻ᵏ)
            eₖ = mod(-1 *  modulo_div(E_of_α⁻ᵏ, dL_of_α⁻ᵏ,p),p)
            push!(ê,eₖ)
        else
            push!(ê,0)
        end
    end
    return reshape(ê,1,n)
end


error_values (generic function with 1 method)

In [128]:
"""
RSNum_Err(L,E,p,α,n,received)

Info:
Function to find the estimated original codeword (ĉ), and the estimated error (ê).

Args:
L: MyPolynomial object representing the Error Locator Polynomial
E: MyPolynomial object representing the Error Evaluator Polynomial
p: The prime number that defines the Finite Field Fₚ
α: The chosen primitive element from Fₚ
n: The length of the code
received: The received codeword, which contains errors

Returns:
ĉ: The estimated original codeword.
ê: The estimated error vector.
"""
function RSNum_Err(L,E,p,α,n,received)
    I = error_locations(L,p,α,n)
    @show I
    ê = error_values(E,I,p,α,n)
    ĉ = [mod(i,p) for i in received - ê]

    return ê,ĉ
end

RSNum_Err

In [129]:
ê,ĉ = RSNum_Err(L,E,p,α,n,received)
@show ê,ĉ

I = Any[5, 7, 13]
(ê, ĉ) = (Any[0 0 0 0 0 11 0 14 0 0 0 0 0 15 0 0], [9 1 15 16 9 5 10 8 14 14 16 12 0 9 0 8])


(Any[0 0 … 0 0], [9 1 … 0 8])

#### RS Decoding [BONUS - EXTRA CREDIT]

As said in the lecture on Monday 29th November, this step of decoding the corrected code is no longer required. However, groups that do it are to receive **EXTRA CREDIT** for doing so.

In [130]:
#========================================================
FUNCTIONS FOR INVERTING A MATRIX UNDER MODULAR ARITHMETIC
========================================================#

function getMatrixMinor(A, i, j)
    m, n = size(A)
    B = similar(A, m-1, n-1)
    for j′=1:j-1, i′=1:i-1; B[i′,j′]=A[i′,j′]; end
    for j′=1:j-1, i′=i+1:m; B[i′-1,j′]=A[i′,j′]; end
    for j′=j+1:n, i′=1:i-1; B[i′,j′-1]=A[i′,j′]; end
    for j′=j+1:n, i′=i+1:m; B[i′-1,j′-1]=A[i′,j′]; end
    return B
end

function getMatrixDeternminant(m,p)
    if length(m) == 4
        return mod(m[1,1]*m[2,2]-m[1,2]*m[2,1],p)
    end
    determinant = 0
    for c in (1:size(m)[2])
        determinant += mod((-1)^(c+1)*m[1,c],p)*getMatrixDeternminant(getMatrixMinor(m,1,c),p)
    end
    return mod(determinant,p)
end

function matrix_from_2D_vector(vec::Vector)
    n = length(vec)
    arr = []
    for i in 1:n
        append!(arr,vec[i])
    end
    return permutedims(reshape(arr,n,n))
end

function getMatrixInverse(m,p)
    determinant = getMatrixDeternminant(m,p)
    if size(m)[1] == 2
        return [[mod(m[2,2]*modulo_div(1,determinant,p),p), mod(-1*m[1,2]*modulo_div(1,determinant,p),p)],
            [mod(-1*m[2,1]*modulo_div(1,determinant,p),p), mod(m[1,1]*modulo_div(1,determinant,p),p)]]
    end 
    #find matrix of cofactors
    cofactors = []
    for r in (1:size(m)[1])
        cofactorRow = []
        for c in (1:size(m)[2])
            minor = getMatrixMinor(m,r,c)
            push!(cofactorRow,mod(((-1)^(r+c))*getMatrixDeternminant(minor,p),p))
        end
        push!(cofactors,cofactorRow)   
    end
    cofactorsM = transpose(matrix_from_2D_vector(cofactors))

    for r in (1:size(cofactorsM)[1])
        for c in (1:size(cofactorsM)[2])
            cofactorsM[r,c] = mod(cofactorsM[r,c]*modulo_div(1,determinant,p),p)
        end
    end
    return cofactorsM
end



getMatrixInverse (generic function with 1 method)

In [131]:
"""
decode_corrected_code(ĉ,p,α,n,k)

Info:
Function to decode a corrected codeword back to a readable message.
This is achieved by creating a square matrix by only taking the first k columns of the k×(n+k) generator matrix,
then multiplying the first k characters of the codeword by the inverse of this matrix.

Args:
ĉ: The estimated original codeword
p: The prime number that defines the Finite Field Fₚ
α: The chosen primitive element from Fₚ
n: The length of the code
k: The length of the message

Returns:
An array containing the decoded message
"""
function decode_corrected_code(ĉ,p,α,n,k)
    # Calculate Generator Matrix
    G = generator_matrix(p,α,n,k)
    G_prime = []
    for i in 1:(k^2)
        push!(G_prime,G[i])
    end
    G_prime = reshape(G_prime,k,k)
    code_prime = reshape([ĉ[i] for i in 1:k],1,k)

    return [mod(i,p) for i in code_prime * getMatrixInverse(G_prime,p)]
end

    

decode_corrected_code

In [132]:
"""
RSNum_Dec(received,p,α,n,k)

Info: 
Function to correct and decode received RS Encoded messages.

Args:
received: The received codeword, which contains errors.
p: The prime number that defines the Finite Field Fₚ
α: The chosen primitive element from Fₚ
n: The length of the code
k: The length of the message

Returns:
m̂: An array containing the decoded message.
"""
function RSNum_Dec(received,p,α,n,k)
    s_vector,s_poly = RSNum_Syndrome(received,p,α,n,k)
    L,E = RSNum_KeyPolys(s_poly,p,α,n,d)
    ê,ĉ = RSNum_Err(L,E,p,α,n,received)
    m̂ = decode_corrected_code(ĉ,p,α,n,k)

    return m̂
end
    
RSNum_Dec(received,p,α,n,k)

L = MyPolynomial{Int64}([11, 2, 14, 1], 3, 17)
E = MyPolynomial{Int64}([5, 16, 11], 2, 17)
I = Any[5, 7, 13]


1×10 Matrix{Int64}:
 7  3  8  7  1  9  4  4  0  0

### 3.1.2 RS Codes on $\mathbb{F}_{p}$

The above code should work well for other choices of $p$, $\alpha$, $[n,k,d]$. Try another $p$ with $30<p<100$ and demonstrate the correctness of your code. 

In [133]:
p = 19
n = 18
k = 10
d = 9

# Get all primitive elements in that Finite Field
prim_els = find_primitive(p)

# Pick a primitive element at random to be α
α = prim_els[rand(1:length(prim_els))]
@show α

m = reshape(collect(1:10),1,10)

codeword = RSNum_Enc(m,p,α,n,k)

error_codeword = RSNum_Channel(codeword,p,n,d)

syn_vec,syn_poly = RSNum_Syndrome(error_codeword,p,α,n,k)

L,E = RSNum_KeyPolys(syn_poly,p,α,n,d)

ê,ĉ = RSNum_Err(L,E,p,α,n,error_codeword)

decode_corrected_code(ĉ,p,α,n,k)


α = 15
I_for_show = [5, 12, 14, 13]
e = Any[0 0 0 0 0 11 0 0 0 0 0 0 14 19 17 0 0 0]
L = MyPolynomial{Int64}([17, 14, 8, 1], 3, 19)
E = MyPolynomial{Int64}([18, 7, 15], 2, 19)
I = Any[5, 12, 14]


1×10 Matrix{Int64}:
 1  2  3  4  5  6  7  8  9  10

## 3.2 RS Codes on $\mathbb{F}_{p^k}$ (50%)

### 3.2.1 RS Codes on $\mathbb{F}_{16}$

Now we consider the finite field $\mathbb{F}_{16} = \mathbb{F}_2[z]/z^4 + z +1$, and an RS code $\mathcal{C} \subset \mathbb{F}_{16}$ with parameters $[15,k,7]$. 

Repeat the steps in 3.1.1 and change the function names from `RSNum_...` to `RSPoly_...`. Provide necessary documentation. 

#### 1. Finding a primitive element $\alpha$

In [134]:
function is_primitive_element(element,poly)
    p = poly.p
    poly = remove_top_zeros(poly)
    element = remove_top_zeros(element)
    q = p^poly.order # size of finite field q = pᵈ

    element_acc = ConstructMyPolynomial(element.coeffs,element.p)
    for t in 2:q
        element_acc = divrem(element_acc * element,poly)[2]
        if divrem(element_acc,poly)[2] == ConstructMyPolynomial([1],p)
            if t == q - 1
                return true
            else
                return false
            end
        end
    end
end

function find_primitive(poly)
    poly = remove_top_zeros(poly)
    p = poly.p
    q = p^poly.order
    prim_els = []

    for i in 1:q-2
        el = mod(ConstructMyPolynomial(append!(zeros(Int,i),[1]),p),poly)
        #print(el,'x')
        if is_primitive_element(el,poly)
            push!(prim_els,el)
        end
    end

    return prim_els
end


find_primitive (generic function with 1 method)

In [135]:
# Define the Finite Field
p = 2
f = ConstructMyPolynomial([1,1,0,0,1],p)

# Get all primitive elements in that Finite Field
prim_els = find_primitive(f)

# Pick a primitive element at random to be α
α = prim_els[rand(1:length(prim_els))]
print(α,'x')

1 + (1x^1) + (0x^2) + (1x^3)


#### 2. What is the value of $k$?

RS Codes are defined by three parameters: **$[n,k,d]$**, where $d = n - k + 1$.

Taking the RS Code given in the question:  **$[16,k,7]$**, we can see that $n = 15$ and $n - k + 1 = 7$. 

We can substitute in $n = 15$ to get: $15 - k + 1 = 7$. Solving this equation yields **$k = 9$**.

In [136]:
# Define the values of [n,k,d]
n = 15
k = 9
d = 7

7

#### 3. RS Encoding

In [137]:
# Helper Functions for RS Encoding

"""
generator_matrix_poly(p,f,α,n,k)

Info:
Creates the generator matrix, G, for RS Encoding using the given arguments.

Args:
p: The prime number that defines the Finite Field Fq
f: The polynomial that defines the Finite Field Fq
α: The chosen primitive element from Fq
n: The length of the code
k: The length of the message

Returns:
A k×n matrix of type Matrix{MyPolynomial{Int}} which is the generator matrix of the RS Code [n,k,d]
"""
function generator_matrix_poly(p,f,α,n,k)
    # First row of the matrix will always be n 1s, so generate that
    gen_mat = reshape(ones_poly(n,p),1,n)
    
    for i in 1:k-1
        # Calculate the defining element for that row (αⁱ, where i goes up to k)
        αⁱ = powermod(α,i,f)
        row = reshape(fill(αⁱ,n),1,n)
        for j in 1:n
            # Mulitply the defining element for the given row by ascending powers up to n
            row[j] = powermod(row[j],j-1,f)
        end
        # Add the new row to the bottom of the matrix
        gen_mat = vcat(gen_mat,row)
    end
    return gen_mat
end


generator_matrix_poly

In [138]:
"""
RSPoly_Enc(message,p,f,α,n,k)

Info:
Encodes the given message using Reed-Solomon Encoding.

Args:
message: Matrix{MyPolynomial{Int}} of dimension 1×k containing the message
p: The prime number that defines the Finite Field Fq
f: The polynomial that defines the Finite Field Fq
α: The chosen primitive element from Fq
n: The length of the code
k: The length of the message

Returns:
A 1×n matrix of type Matrix{MyPolynomial{Int}}, which is the encoded message
"""

function RSPoly_Enc(message::Matrix{MyPolynomial{Int}},p,f,α,n,k)
    if typeof(message) != Matrix{MyPolynomial{Int}}
        println("Please enter message in the format of a 1×k matrix")
        println("Type of message should be Matrix{MyPolynomial{Int}}, not $(typeof(message))")
        return -1
    end
    if length(message) != k
        println("Length of message should be $k, which is the value of k provided")
        return -1
    end

    # Ensure each element in our message is in Fq
    # If not then, perform modulo to make it comply
    message = [mod(i,f) for i in message]
    
    # Calculate the Generator Matrix for the given parameters
    G = generator_matrix_poly(p,f,α,n,k)
    
    # Calculate the code by multiplying the message and the generator matrix
    code = message * G

    code = [mod(i,f) for i in code]

    return code

end

RSPoly_Enc (generic function with 1 method)

In [139]:
# Test: Encode a message

message = [
            ConstructMyPolynomial([0,1],2),
            ConstructMyPolynomial([0,0,1],2),
            ConstructMyPolynomial([0,0,0,1],2),
            ConstructMyPolynomial([0,1,1],2),
            ConstructMyPolynomial([0,0,1,1],2),
            ConstructMyPolynomial([1,0,1],2),
            ConstructMyPolynomial([1,0,0,1],2),
            ConstructMyPolynomial([1,0,1,1],2),
            ConstructMyPolynomial([1,1,1,1],2)
            ]

message = reshape(message,1,k)

codeword = RSPoly_Enc(message,p,f,α,n,k)

1×15 Matrix{MyPolynomial{Int64}}:
 MyPolynomial{Int64}([0, 1, 0, 1], 3, 2)  …  MyPolynomial{Int64}([0, 1, 1, 1], 3, 2)

#### 4. Simulating the Channel

In [140]:
# HELPER FUNCTIONS

# Returns all elements of a Finite Field Fₚ/f(x)
function finite_field_elements(p,f::MyPolynomial)
    # Defining the size of the Finite Field
    q = p^f.order

    # By definition, 0 and 1 are always members of a Finite Field
    elements = [ConstructMyPolynomial([0],2),ConstructMyPolynomial([1],2)]
    for i in 1:q-2
        # We obtain elements by dividing different powers of x by f(x)
        el = divrem(ConstructMyPolynomial(append!(zeros(Int,i),[1]),p),f)[2]
        push!(elements,el)
    end
    return elements
end


finite_field_elements (generic function with 1 method)

In [141]:
"""
RSPoly_Channel(code,p,f,n,d)

Info:
Simulates errors in the channel by adding (d-1)/2 errors, so that we can test our error correction mechanism later.
This is achieved by randomly choosing the position of the errors, and then adding a random number to the code in that position.

Args:
code: Matrix{Int} of dimension 1×n, the codeword to be transmitted accross the channel
p: The prime number that defines the Finite Field Fq
f: The polynomial that defines the Finite Field Fq
n: The length of the code
d: d as defined for the RS Code

Returns:
A 1×n matrix of type Matrix{MyPolynomial{Int}}, which is the codeword with error introduced
"""

function RSPoly_Channel(code,p,f,n,d)

    # q = pᵈ
    q = p^f.order

    # Maximum number of errors
    t::Int = floor((d-1)/2)

    # Find all elements in the Finite Field
    ff_els = finite_field_elements(p,f)

    # Randomly choose t unique locations of the errors
    I = sample((1:n),t,replace=false)
    I_for_show = [i-1 for i in I]
    @show I_for_show
    # Build error vector, which has non-zero elements at eᵢ where i∈I
    e = []
    for i in 1:n
        if i in I
            push!(e,ff_els[rand(1:q)])
        else
            push!(e,ConstructMyPolynomial([0],2))
        end
    end

    # Convert e from type Vector{MyPolynomial} to Matrix{MyPolynomial}
    # This makes it the same type and shape as the code, allowing for addition
    e = reshape(e,1,n)
    # Add error to the code
    changed_code = code + e
    changed_code = [mod(i,f) for i in changed_code]

    return changed_code
end


RSPoly_Channel (generic function with 1 method)

In [142]:
# Test: Transmit codeword through channel
received = RSPoly_Channel(codeword,p,f,n,d)

I_for_show = [10, 13, 12]


1×15 Matrix{MyPolynomial{Int64}}:
 MyPolynomial{Int64}([0, 1, 0, 1], 3, 2)  …  MyPolynomial{Int64}([0, 1, 1, 1], 3, 2)

#### 5. Syndrome Vector and Polynomial

In [143]:
# Helper functions

"""
parity_check_matrix_poly(p,f,α,n,k)

Info:
Creates the parity-check matrix, H, for RS Decoding using the given arguments.

Args:
p: The prime number that defines the Finite Field Fq
f: The polynomial that defines the Finite Field Fq
α: The chosen primitive element from Fq
n: The length of the code
k: The length of the message

Returns:
A (n-k)×n matrix of type Matrix{MyPolynomial} which is the parity-check matrix of the RS Code [n,k,d]

"""

function parity_check_matrix_poly(p,f,α,n,k)
    # Initialise a matrix of zero polynomials with the correct shape: (n-k)×n
    pc_mat::Matrix{MyPolynomial{Int}} = reshape(zeros_poly((n-k)*n,p),n-k,n)
    for i in 1:n-k
        # Calculate the defining element for that row (αⁱ, where i goes up to k)
        αⁱ = powermod(α,i,f)
        row = reshape(fill(αⁱ,n),1,n)
        for j in 1:n
            # Multiply the defining element for the given row by ascending powers up to n
            pc_mat[i+(j-1)*(n-k)] = powermod(row[j],j-1,f)
        end
    end
    return pc_mat
end


parity_check_matrix_poly (generic function with 1 method)

In [144]:
"""
RSPoly_Syndrome(changed_code,p,f,α,n,k)

Info:
Function to calculate the syndrome vector and syndrome polynomial from the changed code.
This is achieved by doing rHᵀ, where r is the received errorred code, and H is the parity check matrix.
r = (c + e), so the operation is actually (c + e)Hᵀ = cHᵀ + eHᵀ (by associativity of matrix multiplication).
cHᵀ = 0 by the properties of the parity-check matrix, leaving us with only eHᵀ, which is the syndrome vector.

Args:
changed_code: The received codeword, which contains errors.
p: The prime number that defines the Finite Field Fq
f: The polynomial that defines the Finite Field Fq
α: The chosen primitive element from Fq
n: The length of the code
k: The length of the message
"""
function RSPoly_Syndrome(changed_code,p,f,α,n,k)
    # Calculate the parity-check Matrix
    H = parity_check_matrix_poly(p,f,α,n,k)

    # Calculate syndrome vector
    syndrome_vector = changed_code * permutedims(H)
    syndrome_vector = [mod(i,f) for i in syndrome_vector]

    # Return the syndrome vector and syndrome polynomial
    return (syndrome_vector,ConstructMyPolynomial(vec(syndrome_vector),p))
end

RSPoly_Syndrome

In [145]:
# Test: Calculate Syndrome Vector/Polynomial from Received Codeword
s_vector,s_poly = RSPoly_Syndrome(received,p,f,α,n,k)
println("Syndrome Vector: ", s_vector)
println("Syndrome Polynomial S(z) = ", string(s_poly,'z'))

Syndrome Vector: MyPolynomial{Int64}[MyPolynomial{Int64}([1, 1, 0, 1], 3, 2) MyPolynomial{Int64}([1, 0, 1, 1], 3, 2) MyPolynomial{Int64}([1, 0, 0, 1], 3, 2) MyPolynomial{Int64}([0, 1, 0, 1], 3, 2) MyPolynomial{Int64}([1], 0, 2) MyPolynomial{Int64}([0, 0, 1], 2, 2)]
Syndrome Polynomial S(z) = 1 + (1z^1) + (0z^2) + (1z^3) + [1 + (0z^1) + (1z^2) + (1z^3)]z^1 + [1 + (0z^1) + (0z^2) + (1z^3)]z^2 + [0 + (1z^1) + (0z^2) + (1z^3)]z^3 + [1]z^4 + [0 + (0z^1) + (1z^2)]z^5


#### 6. Error Locator Polynomial and Error Evaluator Polynomial

In [146]:
# HELPER FUNCTIONS

"""
partial_euclidean_algorithm_poly(a::MyPolynomial{MyPolynomial{Int}},b::MyPolynomial{MyPolynomial{Int}},f::MyPolynomial{Int},stop::Int)

Info:
Helper function that performs the Extended Euclidian Algorithm on two polynomials until a specified stop point.
This stop point refers to the point at which the order of the remainder is ≤ stop. Rewritten for handling polynomials with polynomial coefficients.
"""
function partial_euclidean_algorithm_poly(a::MyPolynomial{MyPolynomial{Int}},b::MyPolynomial{MyPolynomial{Int}},f::MyPolynomial{Int},stop::Int)
    p = a.p
    a = remove_top_zeros(a)
    b = remove_top_zeros(b)

    x0,x1 = ConstructMyPolynomial([ConstructMyPolynomial([1],p)],p),ConstructMyPolynomial([ConstructMyPolynomial([0],p)],p)
    y0,y1 = ConstructMyPolynomial([ConstructMyPolynomial([0],p)],p),ConstructMyPolynomial([ConstructMyPolynomial([1],p)],p)

    r = b

    last_run = false

     while r.order > stop || !last_run
        if r.order <= stop
            last_run = true
        end
        q,r = divrem(a,b,f)
        a = b
        b = r

        x0, x1 = x1, mod(x0 - mod(q*x1,f),f)
        y0, y1 = y1, mod(y0 - mod(q*y1,f),f)
    end
    return (a,x0,y0)
end

partial_euclidean_algorithm_poly

In [147]:
"""
RSPoly_KeyPolys(s_poly,p,f,α,n,d)

Info:
Function to calculate the Error Locator Polynomial (L), and the Error Evaluator Polynomial (E),
from the syndrome polynomial.

Args:
s_poly: Syndrome Polynomial
p: The prime number that defines the Finite Field Fq
f: The polynomial that defines the Finite Field Fq
α: The chosen primitive element from Fq
n: The length of the code
k: The length of the message

Returns:
L: MyPolynomial object representing the Error Locator Polynomial
E: MyPolynomial object representing the Error Evaluator Polynomial
"""
function RSPoly_KeyPolys(s_poly,p,f,α,n,d)
    t::Int = floor((d-1)/2)
    z = ConstructMyPolynomial(append!(zeros_poly(d-1,p),[ConstructMyPolynomial([1],p)]),p)
    r,b,a = partial_euclidean_algorithm_poly(z,s_poly,f,t-1)

    E_prime = divrem(r,z,f)[2]
    L_prime = divrem(a,z,f)[2]

    c_inv = modulo_div(ConstructMyPolynomial([1],p),last(L_prime.coeffs),f)
    L = mod(L_prime * c_inv,f)
    E = mod(E_prime * c_inv,f)

    return L,E
end

RSPoly_KeyPolys

In [148]:
# Test: Calculate the Error Locator and Evaluator Polynomials from the Syndrome Polynomial
L,E = RSPoly_KeyPolys(s_poly,p,f,α,n,d)
println("L: ",string(L,'z'))
println("E: ",string(E,'z'))

L: 1 + (1z^1) + (1z^2) + [1 + (1z^1) + (0z^2) + (1z^3)]z^1 + [1 + (1z^1)]z^2 + [1]z^3
E: 0 + (0z^1) + (1z^2) + [0 + (0z^1) + (1z^2) + (1z^3)]z^1 + [0 + (1z^1)]z^2


#### 7. Estimating Error

In [149]:
# HELPER FUNCTIONS

# Function to find the set of error locations, I.
function error_locations_poly(L,p,f,α,n)
    I = []
    for k in 0:n-1
        # We evaluate L(α⁻ᵏ). If it equals zero we know there is an error at that location.
        α⁻ᵏ = modulo_div(ConstructMyPolynomial([1],p),powermod(α,k,f),f)
        if evaluate(L,α⁻ᵏ,f) == ConstructMyPolynomial([0],p)
            push!(I,k)
        end
    end
    @show I
    return I
end

# Function to find the estimated error vector, ê.
function error_values_poly(E,I,p,f,α,n)
    dL = derivative(L)
    ê = []
    for k in 0:n-1
        if k in I
            α⁻ᵏ = modulo_div(ConstructMyPolynomial([1],p),powermod(α,k,f),f)
            E_of_α⁻ᵏ = evaluate(E,α⁻ᵏ,f)
            dL_of_α⁻ᵏ = evaluate(dL,α⁻ᵏ,f)
            eₖ = mod(-1 *  modulo_div(E_of_α⁻ᵏ, dL_of_α⁻ᵏ,f),f)
            push!(ê,eₖ)
        else
            push!(ê,ConstructMyPolynomial([0],p))
        end
    end
    return reshape(ê,1,n)
end

error_values_poly (generic function with 1 method)

In [150]:
"""
RSPoly_Err(L,E,p,f,α,n,received)

Info:
Function to find the estimated original codeword (ĉ), and the estimated error (ê).

Args:
L: MyPolynomial object representing the Error Locator Polynomial
E: MyPolynomial object representing the Error Evaluator Polynomial
p: The prime number that defines the Finite Field Fq
f: The polynomial that defines the Finite Field Fq
α: The chosen primitive element from Fq
n: The length of the code
received: The received codeword, which contains errors

Returns:
ĉ: The estimated original codeword.
ê: The estimated error vector.
"""
function RSPoly_Err(L,E,p,f,α,n,received)
    I = error_locations_poly(L,p,f,α,n)
    ê = error_values_poly(E,I,p,f,α,n)
    ĉ = [mod(i,f) for i in received - ê]

    return ê,ĉ
end

RSPoly_Err

In [151]:
# Test: Recover the original codeword
ê,ĉ = RSPoly_Err(L,E,p,f,α,n,received)
ĉ == codeword

I = Any[10, 12, 13]


true

#### RS Decoding [BONUS - EXTRA CREDIT]

As said in the lecture on Monday 29th November, this step of decoding the corrected code is no longer required. However, groups that do it are to receive **EXTRA CREDIT** for doing so.

In [152]:
#=======================================================================
FUNCTIONS FOR INVERTING A MATRIX OF POLYNOMIALS UNDER MODULAR ARITHMETIC
=======================================================================#

function getMatrixMinor(A, i, j)
    m, n = size(A)
    B = similar(A, m-1, n-1)
    for j′=1:j-1, i′=1:i-1; B[i′,j′]=A[i′,j′]; end
    for j′=1:j-1, i′=i+1:m; B[i′-1,j′]=A[i′,j′]; end
    for j′=j+1:n, i′=1:i-1; B[i′,j′-1]=A[i′,j′]; end
    for j′=j+1:n, i′=i+1:m; B[i′-1,j′-1]=A[i′,j′]; end
    return B
end

function getMatrixDeternminantPoly(m,f)
    p = m[1].p
    if length(m) == 4
        return mod(m[1,1]*m[2,2]-m[1,2]*m[2,1],f)
    end
    determinant::MyPolynomial{Int64} = ConstructMyPolynomial([0],p)
    for c in (1:size(m)[2])
        determinant = determinant + mod((-1)^(c+1)*m[1,c],f) * getMatrixDeternminantPoly(getMatrixMinor(m,1,c),f)
    end
    return mod(determinant,f)
end

function matrix_from_2D_vector(vec::Vector)
    n = length(vec)
    arr = []
    for i in 1:n
        append!(arr,vec[i])
    end
    return permutedims(reshape(arr,n,n))
end

function getMatrixInversePoly(m,f)
    p = m[1].p
    determinant = getMatrixDeternminantPoly(m,f)
    if size(m)[1] == 2
        return [[mod(m[2,2]*modulo_div(ConstructMyPolynomial([1],p),determinant,f),f), mod(-1*m[1,2]*modulo_div(ConstructMyPolynomial([1],p),determinant,f),f)],
            [mod(-1*m[2,1]*modulo_div(ConstructMyPolynomial([1],p),determinant,f),f), mod(m[1,1]*modulo_div(ConstructMyPolynomial([1],p),determinant,f),f)]]
    end 
    #find matrix of cofactors
    cofactors = []
    for r in (1:size(m)[1])
        cofactorRow = []
        for c in (1:size(m)[2])
            minor = getMatrixMinor(m,r,c)
            push!(cofactorRow,mod(((-1)^(r+c))*getMatrixDeternminantPoly(minor,p),f))
        end
        push!(cofactors,cofactorRow)   
    end
    cofactorsM = permutedims(matrix_from_2D_vector(cofactors))

    for r in (1:size(cofactorsM)[1])
        for c in (1:size(cofactorsM)[2])
            cofactorsM[r,c] = mod(cofactorsM[r,c]*modulo_div(ConstructMyPolynomial([1],p),determinant,f),f)
        end
    end
    return cofactorsM
end


getMatrixInversePoly (generic function with 1 method)

In [153]:
"""
decode_corrected_code_poly(ĉ,p,f,α,n,k)

Info:
Function to decode a corrected codeword back to a readable message.
This is achieved by creating a square matrix by only taking the first k columns of the k×(n+k) generator matrix,
then multiplying the first k characters of the codeword by the inverse of this matrix.

Args:
ĉ: The estimated original codeword
p: The prime number that defines the Finite Field Fq
f: The polynomial that defines the Finite Field Fq
α: The chosen primitive element from Fq
n: The length of the code
k: The length of the message

Returns:
An array containing the decoded message
"""
function decode_corrected_code_poly(ĉ,p,f,α,n,k)
    # Calculate Generator Matrix
    G = generator_matrix_poly(p,f,α,n,k)
    G_prime = []
    for i in 1:(k^2)
        push!(G_prime,G[i])
    end
    G_prime = reshape(G_prime,k,k)
    code_prime = reshape([ĉ[i] for i in 1:k],1,k)

    return [mod(i,f) for i in code_prime * getMatrixInversePoly(G_prime,f)]
end


decode_corrected_code_poly

In [154]:
"""
RSPoly_Dec(received,p,f,α,n,k)

Info: 
Function to correct and decode received RS Encoded messages.

Args:
received: The received codeword, which contains errors.
p: The prime number that defines the Finite Field Fq
f: The polynomial that defines the Finite Field Fq
α: The chosen primitive element from Fq
n: The length of the code
k: The length of the message

Returns:
m̂: An array containing the decoded message.
"""
function RSPoly_Dec(received,p,f,α,n,k)
    s_vector,s_poly = RSPoly_Syndrome(received,p,f,α,n,k)
    L,E = RSPoly_KeyPolys(s_poly,p,f,α,n,d)
    ê,ĉ = RSPoly_Err(L,E,p,f,α,n,received)
    m̂ = decode_corrected_code_poly(ĉ,p,f,α,n,k)

    return m̂
end

RSPoly_Dec

In [155]:
# Test: Correct and decode the recieved code
RSPoly_Dec(received,p,f,α,n,k)

I = Any[10, 12, 13]


1×9 Matrix{MyPolynomial{Int64}}:
 MyPolynomial{Int64}([0, 1], 1, 2)  …  MyPolynomial{Int64}([1, 1, 1, 1], 3, 2)

### 3.2.2 RS Codes on $\mathbb{F}_{2^k}$

The above code should work for other choices of $k$ in $\mathbb{F}_{2^k}$. In 3.2.1, we have worked on the case where $k=4$. Try a different $k$ with $5 \le k \le 10$. 

In [156]:
p = 2
n = 256
k = 200
d = 57
f = ConstructMyPolynomial([1,0,1,1,1,0,0,0,1],2)

# Get all primitive elements in that Finite Field
prim_els = find_primitive(f)

# Pick a primitive element at random to be α#
α = prim_els[rand(1:length(prim_els))]
print(α,'x')

0 + (1x^1) + (1x^2) + (1x^3) + (1x^4) + (1x^5) + (1x^6) + (1x^7)


In [157]:
# Finite field F(2⁶)

p = 2
n = 63
k = 40
d = 24
f = ConstructMyPolynomial([1,1,0,1,1,0,1],2)

# Get all primitive elements in that Finite Field
prim_els = find_primitive(f)

# Pick a primitive element at random to be α
α = prim_els[rand(1:length(prim_els))]
print(α,'x')

function get_message()
    m1 = fill(ConstructMyPolynomial([0,1,1],2),k)
    m2::Matrix{MyPolynomial{Int64}} = reshape(m1,1,k)
    return m2
end

message = get_message()

codeword = RSPoly_Enc(message,p,f,α,n,k)
error_codeword = RSPoly_Channel(codeword,p,f,n,d)
syn_vec,syn_poly = RSPoly_Syndrome(error_codeword,p,f,α,n,k)
L,E = RSPoly_KeyPolys(syn_poly,p,f,α,n,d)
ê,ĉ = RSPoly_Err(L,E,p,f,α,n,error_codeword)

ĉ == codeword

0 + (1x^1) + (1x^2) + (1x^3) + (1x^4) + (1x^5)
I_for_show = [15, 12, 20, 49, 9, 51, 18, 27, 17, 62, 23]
I = Any[9, 12, 15, 17, 18, 20, 23, 49, 51, 62]


true

In [158]:
# Finite field F(2⁸)

p = 2
n = 255
k = 240
d = 16
f = ConstructMyPolynomial([1,0,1,1,1,0,0,0,1],2)

# Get all primitive elements in that Finite Field
prim_els = find_primitive(f)

# Pick a primitive element at random to be α
α = prim_els[rand(1:length(prim_els))]
print(α,'x')

function get_message()
    m1 = fill(ConstructMyPolynomial([0,1,1],2),k)
    m2::Matrix{MyPolynomial{Int64}} = reshape(m1,1,k)
    return m2
end

message = get_message()

codeword = RSPoly_Enc(message,p,f,α,n,k)
error_codeword = RSPoly_Channel(codeword,p,f,n,d)
syn_vec,syn_poly = RSPoly_Syndrome(error_codeword,p,f,α,n,k)
L,E = RSPoly_KeyPolys(syn_poly,p,f,α,n,d)
ê,ĉ = RSPoly_Err(L,E,p,f,α,n,error_codeword)

ĉ == codeword

1 + (1x^1) + (0x^2) + (1x^3) + (1x^4) + (1x^5) + (0x^6) + (1x^7)
I_for_show = [82, 105, 124, 41, 40, 186, 172]
I = Any[40, 41, 82, 105, 124, 172, 186]


true

In [159]:
# Finite field F(2¹⁰)

p = 2
n = 1023
k = 990
d = 34
f = ConstructMyPolynomial([1,0,0,1,0,0,0,0,0,0,1],2)

# Get all primitive elements in that Finite Field
prim_els = find_primitive(f)

# Pick a primitive element at random to be α
α = prim_els[rand(1:length(prim_els))]
print(α,'x')

0 + (0x^1) + (1x^2) + (1x^3) + (1x^4) + (1x^5)


In [160]:
# # Finite field F(2¹⁰)

# p = 2
# n = 1023
# k = 990
# d = 34
# f = ConstructMyPolynomial([1,0,0,1,0,0,0,0,0,0,1],2)

# α = ConstructMyPolynomial([1,0,1,1,1,1,1,1,1],2)

# function get_message()
#     m1 = fill(ConstructMyPolynomial([0,1,1],2),k)
#     m2::Matrix{MyPolynomial{Int64}} = reshape(m1,1,k)
#     return m2
# end

# message = get_message()

# @show codeword = RSPoly_Enc(message,p,f,α,n,k)
# @show error_codeword = RSPoly_Channel(codeword,p,f,n,d)
# syn_vec,syn_poly = RSPoly_Syndrome(error_codeword,p,f,α,n,k)
# L,E = RSPoly_KeyPolys(syn_poly,p,f,α,n,d)
# ê,ĉ = RSPoly_Err(L,E,p,f,α,n,error_codeword)

# ĉ == codeword

## Highlight

Please list a couple of highlights of your coursework that may impress your markers.